In [543]:

from datetime import datetime  # calculating time until expiration
import numpy as np  # performing numerical methods
import pandas as pd  # printing dataframes
import yfinance as yf # retrieving financial data


In [544]:

# parent/super class
"""
Pricing of stock futures and construction of a portfolio containing both underlying equity
and futures as a hedge.

Attributes
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

ticker: str
    ticker of underlying stock

spot_price: float
    spot price of underlying stock, in united states dollars $

initial_time: str
    current date

final_time: str
    expiration date

dividend: float
    dividend yield, in percentage %, default = 0.00%

risk_tolerance: float
    risk tolerance, in percentage %, default = 0.00%

cap: float
    user capital, in united states dollars $, default = $10,000

shares: float
    user position in underlying stock, in united states dollars $, default = 100 * s
year_time: date
    proportion of diff in time 
liquid: float
    Liquid/capital money (user inputted)
    

"""

class Edward_Dashboard:
    def __init__(self, ticker: str, final_time: str, r_tol: float = 0.00,liquid: float=1000, cap: float = 10000, shares: float = 100):
        assert (1 <= len(ticker) <= 5),  'ticker symbol must have between one and five characters'
        assert r_tol >= 0, 'risk tolerance cannot be less than 0.00%'
        assert cap >= 10000,  'user must have at least $10,000 in capital'
        assert shares >= 0.00, 'user must have at least 100 shares in underlying equity'
        self.ticker_symbol = ticker
        # ticker symbol(user inputted )
        self.ticker = yf.Ticker(ticker.upper()).info
        self.tickerObj =yf.Ticker(ticker.upper())
        # spot price
        s = float(self.ticker['regularMarketPrice'])
        assert s >= 0.00,  'stock price cannot be less than $0.00, cannot use this stock'
        self.spot_price = s
    
        # intial time(not user inputted )
        current_date = datetime.now()
        time = f"{current_date.year}-{current_date.month}-{current_date.day}"
        self.initial_time = datetime.strptime(time, "%Y-%m-%d")
        # final time(user inputted)
        self.final_time = datetime.strptime(final_time, "%Y-%m-%d")
        print(self.final_time)
        # porpotion of diff in time
        self.year_time = (self.final_time - self.initial_time).days/365
        print(self.year_time)
        # risk tolerance
        self.risk_tolerance = float(r_tol)
        # liquid/captial money(user inputted)
        self.liquid = float(liquid)
        # total value of shares (user inputted)
        self.shares = float(shares * self.spot_price)
        #total capital
        self.cap = float(cap)
    def get_historical_data(self):
        temp_csv_str = f"{self.ticker_symbol}.csv"
        temp_csv = self.tickerObj.history(period ="max").to_csv(temp_csv_str)
        new_history_df = pd.read_csv(temp_csv_str)
        historical_data_close= new_history_df[["Date","Close"]]
        return historical_data_close
    def div_rate(self):
        #this is the dividend rate from the info of the function
        #this is the dividend rate from the most recent year
        dividend_list = self.tickerObj.dividends.to_list()
        if len(dividend_list) != 0:
            return dividend_list[-1]
        
    def risk_free_rate(self): # calculate risk-free rate of return
        rates = pd.read_csv('rates.csv')

        if 0 < self.year_time <= 1 / 12:
            nominal = rates['1 Mo'].mean()
        elif 1 / 12 < self.year_time <= 1 / 6:
            nominal = rates['2 Mo'].mean()
        elif 1 / 6 < self.year_time <= 1 / 4:
            nominal = rates['3 Mo'].mean()
        elif 1 / 4 < self.year_time <= 1 / 3:
            nominal = rates['4 Mo'].mean()
        elif 1 / 3 < self.year_time <= 1 / 2:
            nominal = rates['6 Mo'].mean()
        elif 1 /  2 < self.year_time <= 1:
            nominal = rates['1 Yr'].mean()
        elif 1 < self.year_time<= 2:
            nominal = rates['2 Yr'].mean()
        elif 2 < self.year_time <= 3:
            nominal = rates['3 Yr'].mean()
        elif 3 < self.year_time <= 5:
            nominal = rates['5 Yr'].mean()
        elif 5 < self.year_time <= 7:
            nominal = rates['7 Yr'].mean()
        elif 7 < self.year_time <= 10:
            nominal = rates['10 Yr'].mean()
        elif 10 < self.year_time <= 20:
            nominal = rates['20 Yr'].mean()
        elif 20 < self.year_time <= 30:
            nominal = rates['30 Yr'].mean()

        return (1 + nominal) / (1 + 0.077) - 1

    def benchmark_rate(self): # calculate performance of a benchmark
        # set the S&P as the default benchmark
        gspc = yf.Ticker('^GSPC').info
        print(gspc)
        print(self.initial_time - (self.year_time * 365))
        hist_table = yf.download(gspc, self.initial_time-(self.year_time*365))['Adj Close']
        hist_s = hist_table.loc[self.initial_time-(self.year_time*365)][gspc]
        benchmark_rate = abs(gspc['regularMarketPrice']- hist_s)/hist_s
        return benchmark_rate

    def pricing(self,time: float,price: float): # pricing of a singular futures contract on underlying equity
        if self.div_rate() is None:
            div_rate = 0
        else:
            div_rate = self.div_rate()
            
        return 100 * price * np.e**(self.risk_free_rate() - div_rate*time)

    def beta(self): # covariance function
        #get the historical data beginning to end
        historical_data = self.get_historical_data()
        hist_price = [price for price in historical_data["Close"]]
        future_price = []
        for i in range(len(hist_price)):
            future_price.append(self.pricing(time = 1/52, price = hist_price[i]))
        return np.cov(hist_price,future_price)
            


    def expected_return(self): # calculate the expected return (w/ respect to a benchmark)
        return float(self.risk_free_rate() + self.beta()*(self.benchmark_rate() - self.risk_free_rate()))

    def n(self): # calculate the ideal number of futures contracts on the underlying equity
        return float(self.beta() * self.shares / self.pricing(1/52,self.spot_price))

    def portfolio(self): # construct an ideal portfolio, containing the user's existing
        # position in the underlying equity and n futures contracts
        data = [("stock", self.shares / self.spot_price, "$" + str(self.shares)),
                    ("futures", self.n(), "$" + str(self.n() * self.pricing(1/52,self.spot_price)))]

        portfolio = pd.DataFrame(data, columns = ["Instrument", " Qty", "Value"])

        return portfolio

    def basis_risk(self): # calculate basis risk of portfolio
        return float(self.n() * (self.spotprice - self.pricing(1/52,self.spot_price)))

    def metrics(self): # calculate performance & risk metric of portfolio
        """"
            add implementation
        """
        pass;

    def _str_(self):
        return f"risk-free-rate{self.risk_free_rate()}, benchmark rate {self.benchmark_rate()}, pricing {self.pricing(1/52,self.spot_price)}, beta {self.beta()}, expect_return {self.expected_return()}, number of future contracts (100 share per contract) {self.n()}, protfolio {self.portfolio()}, basis risk {self.basis_risk()}, metrics {self.metrics()}"
        pass



In [545]:
#practicing with microsoft stock 
 #stock1 = Edward_Dashboard("MSFT", s: float,  tf: str, tol: float = 0.00, cap: float = 10000, shares: float = 100)


In [546]:
msft= Edward_Dashboard("msft","2022-11-27",300)

2022-11-27 00:00:00
0.005479452054794521


In [547]:
msft.ticker_symbol

'msft'

In [526]:
msft.spot_price

247.49

In [527]:
msft.cap

10000.0

In [528]:
msft.initial_time

datetime.datetime(2022, 11, 25, 0, 0)

In [529]:
msft.final_time

datetime.datetime(2022, 11, 27, 0, 0)

In [530]:
msft.year_time

0.005479452054794521

In [531]:
msft.risk_tolerance

300.0

In [532]:
msft.liquid

1000.0

In [533]:
msft.tickerObj

yfinance.Ticker object <MSFT>

In [534]:
msft.div_rate()

0.68

In [535]:
msft.risk_free_rate()

1.2135393690366656

In [536]:
msft.get_historical_data()

,Date,Close
0,1986-03-13 00:00:00-05:00,0.060809
1,1986-03-14 00:00:00-05:00,0.062980
2,1986-03-17 00:00:00-05:00,0.064067
3,1986-03-18 00:00:00-05:00,0.062437
4,1986-03-19 00:00:00-05:00,0.061351
...,...,...
9248,2022-11-18 00:00:00-05:00,241.220001
9249,2022-11-21 00:00:00-05:00,242.050003
9250,2022-11-22 00:00:00-05:00,245.029999
9251,2022-11-23 00:00:00-05:00,247.580002


In [537]:
beta_array = msft.beta()

In [538]:
beta_array

array([[4.39942402e+03, 1.46133585e+06],
       [1.46133585e+06, 4.85405009e+08]])